# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [1]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [2]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

## Main Goal for Week of 6/27:
Adjust the `template`, `integrator`, `cross_correlation`, and `fs_search` functions to take a frequency array as input and output the proper result.
#### Tasks: 
* The main goal is to find the global maximum of the cross-correlation and apply its index to the frequency array to find the corresponding value.

In [47]:
def frequency_time_search(del_T_0, t_start, t_max, 
             data_time_series, a, f, sigma, t_duration, del_T):
    
    times, C = san.cross_correlation(del_T_0, t_start, t_max, data_time_series, 
                                 a, f, sigma, t_duration, del_T)
    C_array = np.array(C)
    C_max_value = np.max(C_array)
    find_index = np.where(C_max_value == C_array)
    
    index_row = int(find_index[0])
    index_column = int(find_index[1])
    
    time_correct = times[index_row]
    frequency_correct = f[index_column]
    
    return time_correct, frequency_correct

In [48]:
a = 1.5
f = np.array([1, 2, 3])   # actual values/size to be determined
sigma = 0.1
t_0 = 2.01
t_start = 2
t_duration = 8
t_max = 20
data_time_series = [Time1, Position1]
data_time_stamps = Time1
del_T_0 = 0.012
del_T = 0.012

In [49]:
search = frequency_search(del_T_0, t_start, t_max, data_time_series, a, f, sigma, t_duration, del_T)

In [50]:
search

(5.15599999999996, 1)

In [30]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

x = np.max(a)
z = np.where(x == a)

In [44]:
int(a[z])

9

In [45]:
int(z[0])

2

In [46]:
int(z[1])

2

In [ ]:
# np.where outputs a tuple: row, column